# READ df_shipto_master

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import math
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import warnings
warnings.filterwarnings('ignore')

---


Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Viewing data

In [ ]:
print("🚀 STARTING BASELINE COSTING ANALYSIS WITH K-MEANS CLUSTERING")
print("="*80)

file_path = '/content/drive/MyDrive/0027. CUỘC THI/01. LOGAGE 2025/02. VÒNG 2/PROJECT/02. IMPLEMENTATION/03. New distribution network design/processed_shipto_master.csv'
df_shipto_master = pd.read_csv(file_path)

print(f"📊 Total ShipToID count: {len(df_shipto_master)}")
print(f"📊 Total KG demand: {df_shipto_master['Annual_KG_Demand_ShipTo'].sum():,.0f}")

🚀 STARTING BASELINE COSTING ANALYSIS WITH K-MEANS CLUSTERING
📊 Total ShipToID count: 486
📊 Total KG demand: 10,736,837


In [ ]:
display(df_shipto_master.head())

,ShipToID,Longitude,Latitude,State,City,Suburb,Annual_CBM_Demand_ShipTo,Annual_KG_Demand_ShipTo,Total_Shipments
0,3001BR,151.001259,-33.797556,New South Wales,Parramatta,North Parramatta,91.182518,11801.9360,313
1,3002BR,149.206300,-35.347000,New South Wales,Queanbeyan,Queanbeyan,100.414742,12320.3690,314
2,3003BR,147.343011,-35.122016,New South Wales,Wagga Wagga,Wagga Wagga,882.881842,99185.9602,694
3,3004BR,150.850020,-33.760995,New South Wales,Blacktown,Glendenning,30.670602,5160.5740,224
4,3005BR,153.129468,-30.287126,New South Wales,Coffs Harbour,Coffs Harbour,149.023835,18935.3500,418


---
# Divided into 5 phases for delivery

In [ ]:
df_shipto_master['Annual_CBM_Demand_ShipTo'] = df_shipto_master['Annual_CBM_Demand_ShipTo'] / 5
df_shipto_master['Annual_KG_Demand_ShipTo'] = df_shipto_master['Annual_KG_Demand_ShipTo'] / 5
df_shipto_master['Total_Shipments'] = df_shipto_master['Total_Shipments'] / 5

---
# Current DC parameters

In [ ]:
# Current DC parameters
lat_ref = -29.79165152  # NSW DC latitude
lon_ref = 151.1250996   # NSW DC longitude
state_dc_curr = "New South Wales"
avg_inv_cbm_curr = 14898
storage_cost_cbm_curr = 30
handling_cost_cbm_curr = 10
TRUCK_CAPACITY = 20000  # 20 tons

print(f"🏢 Current DC: ({lon_ref}, {lat_ref}) in {state_dc_curr}")
print(f"🚛 Truck capacity: {TRUCK_CAPACITY:,} KG")

🏢 Current DC: (151.1250996, -29.79165152) in New South Wales
🚛 Truck capacity: 20,000 KG


---

# Haversine distance

In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate Haversine distance between 2 points (km)"""
    R = 6371  # Earth radius (km)
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Calculate distances from DC to all ShipTo locations
df_shipto_master['distance'] = df_shipto_master.apply(
    lambda row: haversine_distance(lat_ref, lon_ref, row['Latitude'], row['Longitude']),
    axis=1
)

In [ ]:
df_shipto_master['distance']

,distance
0,445.589848
1,643.319302
2,690.662110
3,442.135503
4,200.647763
...,...
481,2844.463442
482,2846.996298
483,2843.808291
484,2831.025593


---
# Performing K-Means Clustering with 8 clusters

In [ ]:
# Prepare data for clustering
X = df_shipto_master[['Longitude', 'Latitude']]

# Perform K-Means with 8 clusters
kmeans = KMeans(n_clusters=8, random_state=42, n_init=10)
df_shipto_master['Cluster'] = kmeans.fit_predict(X)

# Get cluster centers
cluster_centers = kmeans.cluster_centers_

print("✅ K-Means clustering completed")
print("📊 Statistics by cluster:")
for i in range(8):
    cluster_data = df_shipto_master[df_shipto_master['Cluster'] == i]
    print(f"   Cluster {i}: {len(cluster_data)} ShipToID, Total KG: {cluster_data['Annual_KG_Demand_ShipTo'].sum():,.0f}")

print("\n📈 Creating clustering visualization...")

✅ K-Means clustering completed
📊 Statistics by cluster:
   Cluster 0: 152 ShipToID, Total KG: 829,958
   Cluster 1: 40 ShipToID, Total KG: 241,326
   Cluster 2: 87 ShipToID, Total KG: 347,674
   Cluster 3: 135 ShipToID, Total KG: 482,088
   Cluster 4: 22 ShipToID, Total KG: 58,242
   Cluster 5: 39 ShipToID, Total KG: 141,940
   Cluster 6: 3 ShipToID, Total KG: 21,331
   Cluster 7: 8 ShipToID, Total KG: 24,809

📈 Creating clustering visualization...


---
## Main clustering map

In [ ]:
fig_main = go.Figure()

colors = px.colors.qualitative.Set1
for i in range(8):
    cluster_data = df_shipto_master[df_shipto_master['Cluster'] == i]

    # Plot ShipToIDs in cluster
    fig_main.add_trace(go.Scatter(
        x=cluster_data['Longitude'],
        y=cluster_data['Latitude'],
        mode='markers',
        marker=dict(
            color=colors[i],
            size=cluster_data['Annual_KG_Demand_ShipTo']/5000,  # Scale size by demand
            sizemin=4,
            # Removed sizemax - not a valid property
            opacity=0.7,
            line=dict(width=1, color='white')
        ),
        text=[f"<b>{row['ShipToID']}</b><br>Cluster: {row['Cluster']}<br>KG/year: {row['Annual_KG_Demand_ShipTo']:,.0f}<br>Distance: {row['distance']:.1f}km"
              for _, row in cluster_data.iterrows()],
        hovertemplate='%{text}<extra></extra>',
        name=f'Cluster {i} ({len(cluster_data)} points)'
    ))

# Plot cluster centers
fig_main.add_trace(go.Scatter(
    x=cluster_centers[:, 0],
    y=cluster_centers[:, 1],
    mode='markers+text',
    marker=dict(symbol='star', size=15, color='black', line=dict(width=2, color='white')),
    text=[f'C{i}' for i in range(8)],
    textposition="middle center",
    textfont=dict(color='white', size=12),
    name='Cluster Centers',
    hovertemplate='Cluster Center %{text}<extra></extra>'
))

# Plot current DC
fig_main.add_trace(go.Scatter(
    x=[lon_ref],
    y=[lat_ref],
    mode='markers+text',
    marker=dict(symbol='diamond', size=20, color='red'),
    text=['Current DC'],
    textposition="top center",
    name='Current DC (NSW)',
    hovertemplate='Current DC<br>NSW<extra></extra>'
))

fig_main.update_layout(
    title="<b>K-Means Clustering (8 Clusters) - Australia Distribution Network</b>",
    xaxis_title="Longitude (°E)",
    yaxis_title="Latitude (°S)",
    height=700,
    showlegend=True,
    legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02)
)

fig_main.show()

---
## Cluster statistics dashboard

In [ ]:
cluster_stats = df_shipto_master.groupby('Cluster').agg({
    'ShipToID': 'count',
    'Annual_KG_Demand_ShipTo': ['sum', 'mean'],
    'distance': ['mean', 'max', 'min']
}).round(2)

cluster_stats.columns = ['Count', 'Total_KG', 'Avg_KG', 'Avg_Distance', 'Max_Distance', 'Min_Distance']
cluster_stats = cluster_stats.reset_index()

# Create dashboard
fig_dashboard = make_subplots(
    rows=2, cols=2,
    subplot_titles=['ShipToID Count by Cluster', 'Total KG Demand by Cluster',
                   'Average Distance from DC', 'KG Demand Distribution'],
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "box"}]]
)

# Count by cluster
fig_dashboard.add_trace(
    go.Bar(x=cluster_stats['Cluster'], y=cluster_stats['Count'],
           marker_color=colors[:8], name='Count'),
    row=1, col=1
)

# Total KG by cluster
fig_dashboard.add_trace(
    go.Bar(x=cluster_stats['Cluster'], y=cluster_stats['Total_KG'],
           marker_color=colors[:8], name='Total KG'),
    row=1, col=2
)

# Average distance
fig_dashboard.add_trace(
    go.Bar(x=cluster_stats['Cluster'], y=cluster_stats['Avg_Distance'],
           marker_color=colors[:8], name='Avg Distance'),
    row=2, col=1
)

# Box plot demand distribution
for i in range(8):
    cluster_data = df_shipto_master[df_shipto_master['Cluster'] == i]
    fig_dashboard.add_trace(
        go.Box(y=cluster_data['Annual_KG_Demand_ShipTo'], name=f'Cluster {i}',
               marker_color=colors[i]),
        row=2, col=2
    )

fig_dashboard.update_layout(height=800, showlegend=False,
                           title_text="<b>K-Means Clustering Statistics Dashboard</b>")
fig_dashboard.show()

---
# CONSOLIDATION ALGORITHM AND TSP FOR EACH CLUSTER

In [ ]:
def split_by_capacity(data_cluster, capacity):
    """Split cluster into truck trips by capacity"""
    remaining_data = data_cluster.copy()
    trucks = []

    while remaining_data['Annual_KG_Demand_ShipTo'].sum() > 0:
        truck_load = pd.DataFrame(columns=remaining_data.columns)
        current_weight = 0
        remaining_data = remaining_data.sort_values(by='Annual_KG_Demand_ShipTo', ascending=False)

        for index, row in remaining_data.iterrows():
            if current_weight + row['Annual_KG_Demand_ShipTo'] <= capacity:
                truck_load = pd.concat([truck_load, pd.DataFrame([row])])
                current_weight += row['Annual_KG_Demand_ShipTo']
                remaining_data = remaining_data[remaining_data['ShipToID'] != row['ShipToID']]
            elif row['Annual_KG_Demand_ShipTo'] > capacity:
                # Split if demand > capacity
                new_row = row.copy()
                new_row['Annual_KG_Demand_ShipTo'] = capacity - current_weight
                truck_load = pd.concat([truck_load, pd.DataFrame([new_row])])
                remaining_data.loc[index, 'Annual_KG_Demand_ShipTo'] -= new_row['Annual_KG_Demand_ShipTo']
                break

        if len(truck_load) > 0:
            trucks.append(truck_load)

    return trucks


def nearest_neighbor_tsp(dist_matrix, truck_indices):
    """Nearest Neighbor TSP algorithm"""
    n = len(truck_indices)
    if n <= 1:
        return [truck_indices[0], truck_indices[0]] if n == 1 else []

    unvisited = set(range(1, n))  # Start from DC (index 0)
    route = [0]
    current = 0

    while unvisited:
        # Find the nearest unvisited point
        next_point_relative_idx = min(unvisited, key=lambda relative_idx: dist_matrix[current][relative_idx])
        route.append(next_point_relative_idx)
        unvisited.remove(next_point_relative_idx)
        current = next_point_relative_idx

    route.append(0)  # Return to DC
    return [truck_indices[i] for i in route]

# Cost curve for NSW
cost_curve_nsw = {
    'intercept': 360.4,
    'slope': 1.1
}

---
# Starting cluster-by-cluster analysis

In [ ]:
print("\n🚛 Starting cluster-by-cluster analysis...")

all_cluster_results = []
total_baseline_cost = 0

for cluster_id in range(8):
    print(f"\n--- CLUSTER {cluster_id} ---")

    # Get cluster data
    data_cluster = df_shipto_master[df_shipto_master['Cluster'] == cluster_id].copy()

    if len(data_cluster) == 0:
        continue

    # Add DC at the beginning
    dc_row = pd.DataFrame({
        'ShipToID': ['DC'],
        'Longitude': [lon_ref],
        'Latitude': [lat_ref],
        'State': [state_dc_curr],
        'City': ['NSW_DC'],
        'Suburb': ['NSW_DC'],
        'Annual_CBM_Demand_ShipTo': [0],
        'Annual_KG_Demand_ShipTo': [0],
        'Total_Shipments': [0],
        'distance': [0],
        'Cluster': [cluster_id]
    })

    data_with_dc = pd.concat([dc_row, data_cluster], ignore_index=True)


    # Create distance matrix
    n_points = len(data_with_dc)
    dist_matrix = np.zeros((n_points, n_points))

    for i in range(n_points):
        for j in range(n_points):
            if i != j:
                dist_matrix[i][j] = haversine_distance(
                    data_with_dc.iloc[i]['Latitude'], data_with_dc.iloc[i]['Longitude'],
                    data_with_dc.iloc[j]['Latitude'], data_with_dc.iloc[j]['Longitude']
                )

    # Split cluster into truck trips
    trucks = split_by_capacity(data_with_dc, TRUCK_CAPACITY)

    print(f"   Number of truck trips: {len(trucks)}")
    print(f"   Total cluster KG: {data_with_dc['Annual_KG_Demand_ShipTo'].sum():,.0f}")

    cluster_results = []
    cluster_total_cost = 0

    for truck_idx, truck in enumerate(trucks):
        # Find indices of points in data_with_dc for this truck (+ DC)
        indices = [0]  # DC is always at index 0 in data_with_dc
        for _, shipto in truck.iterrows():
            # Find the index of the ShipToID in the data_with_dc DataFrame
            shipto_idx_in_data_with_dc = data_with_dc[data_with_dc['ShipToID'] == shipto['ShipToID']].index
            if not shipto_idx_in_data_with_dc.empty:
                indices.append(shipto_idx_in_data_with_dc[0])
            else:
                print(f"Warning: ShipToID {shipto['ShipToID']} not found in data_with_dc. Skipping.")


        if len(indices) <= 1:
             print(f"Warning: Truck {truck_idx + 1} has only DC or no valid ShipToIDs. Skipping TSP.")
             continue


        # Create sub distance matrix for TSP
        # The indices list contains the absolute indices from data_with_dc
        sub_dist_matrix_for_tsp = dist_matrix[np.ix_(indices, indices)]


        # Apply TSP using indices relative to the sub_dist_matrix
        # The TSP algorithm expects indices from 0 to len(indices)-1
        relative_indices_for_tsp = list(range(len(indices)))
        route_relative_indices = nearest_neighbor_tsp(sub_dist_matrix_for_tsp, relative_indices_for_tsp)

        # Map relative indices back to original ShipToIDs
        route = [data_with_dc.iloc[indices[i]]['ShipToID'] for i in route_relative_indices]


        # Calculate total distance of the route (sum of segments)
        total_distance = 0
        # Iterate through the route and sum distances between consecutive points
        # The route includes the return to DC (e.g., DC -> A -> B -> DC)
        # So we sum distances from the first point up to the second to last point to the last point
        for i in range(len(route_relative_indices) - 1):
            from_point_relative_idx = route_relative_indices[i]
            to_point_relative_idx = route_relative_indices[i+1]
            total_distance += sub_dist_matrix_for_tsp[from_point_relative_idx][to_point_relative_idx]


        # Calculate cost based on the total distance
        # Assuming LTL rate applies for distances > 55km as per the simplified curve logic
        # For a more accurate model, you'd check segments against FTL/LTL distance ranges
        cost = cost_curve_nsw['intercept'] + cost_curve_nsw['slope'] * total_distance


        truck_result = {
            'cluster_id': cluster_id,
            'truck_id': truck_idx + 1,
            'route': ' -> '.join(route), # Store route as a string
            'total_weight': truck['Annual_KG_Demand_ShipTo'].sum(),
            'total_distance': total_distance,
            'cost': cost,
            'num_stops': len(truck)
        }
        cluster_results.append(truck_result)

        print(f"      Truck {truck_idx + 1}: {len(truck)} stops, {truck['Annual_KG_Demand_ShipTo'].sum():,.0f} KG, {total_distance:.1f} km, ${cost:,.0f}")

    cluster_summary = {
        'cluster_id': cluster_id,
        'num_shiptos': len(data_cluster),
        'total_kg': data_cluster['Annual_KG_Demand_ShipTo'].sum(),
        'num_trucks': len(trucks),
        'total_cost': cluster_total_cost, # This needs to be summed up from truck costs
        'avg_distance': data_cluster['distance'].mean(),
        'truck_results': cluster_results
    }

    # Sum up the total cost for the cluster from the truck results
    cluster_summary['total_cost'] = sum(t['cost'] for t in cluster_results)


    all_cluster_results.append(cluster_summary)
    total_baseline_cost += cluster_summary['total_cost'] # Sum cluster total costs

    print(f"   Cluster {cluster_id} total cost: ${cluster_summary['total_cost']:,.0f}")


🚛 Starting cluster-by-cluster analysis...

--- CLUSTER 0 ---
   Number of truck trips: 42
   Total cluster KG: 829,958
      Truck 1: 1 stops, 20,000 KG, 2105.1 km, $2,676
      Truck 2: 1 stops, 20,000 KG, 2105.1 km, $2,676
      Truck 3: 1 stops, 20,000 KG, 2105.1 km, $2,676
      Truck 4: 1 stops, 20,000 KG, 2105.1 km, $2,676
      Truck 5: 1 stops, 20,000 KG, 2128.3 km, $2,702
      Truck 6: 1 stops, 20,000 KG, 2105.1 km, $2,676
      Truck 7: 1 stops, 20,000 KG, 2196.1 km, $2,776
      Truck 8: 1 stops, 20,000 KG, 2128.3 km, $2,702
      Truck 9: 1 stops, 20,000 KG, 2087.3 km, $2,656
      Truck 10: 3 stops, 19,962 KG, 2115.4 km, $2,687
      Truck 11: 2 stops, 19,976 KG, 3218.1 km, $3,900
      Truck 12: 2 stops, 19,995 KG, 2147.3 km, $2,722
      Truck 13: 2 stops, 19,953 KG, 2236.9 km, $2,821
      Truck 14: 3 stops, 19,893 KG, 2250.0 km, $2,835
      Truck 15: 2 stops, 19,796 KG, 2167.3 km, $2,744
      Truck 16: 2 stops, 19,915 KG, 2127.1 km, $2,700
      Truck 17: 2 stops, 

---
# CALCULATE THE TOTAL COMPREHENSIVE COSTS FOR THE 5 PHASES

In [ ]:
print(f"\n💰 CALCULATING COMPREHENSIVE COSTS...")

# Branch Delivery Cost
# This is already calculated as the sum of all truck costs
total_branch_delivery_cost = total_baseline_cost * 5 # TOTAL COMPREHENSIVE COSTS FOR THE 5 PHASES
print(f"🚛 Branch Delivery Cost: ${total_branch_delivery_cost:,.0f}")

total_annual_cbm_outbound = df_shipto_master['Annual_CBM_Demand_ShipTo'].sum() * 5 # TOTAL COMPREHENSIVE COSTS FOR THE 5 PHASES
total_handling_cost = total_annual_cbm_outbound * handling_cost_cbm_curr
print(f"📦 Total Annual CBM: {total_annual_cbm_outbound:,.0f}")
print(f"📦 Handling Cost: ${total_handling_cost:,.0f}")

# Storage Cost
# This comes directly from the CurrentScenario data
total_storage_cost = avg_inv_cbm_curr * storage_cost_cbm_curr
print(f"🏬 Storage Cost: ${total_storage_cost:,.0f}")

# Transfer Cost (= 0 for baseline)
total_transfer_cost = 0
print(f"🔄 Transfer Cost: ${total_transfer_cost:,.0f}")

# Baseline Total Cost - Sum of all cost components
baseline_total_cost = total_branch_delivery_cost + total_handling_cost + total_storage_cost + total_transfer_cost
print(f"\n🎯 BASELINE TOTAL COST: ${baseline_total_cost:,.0f}")


💰 CALCULATING COMPREHENSIVE COSTS...
🚛 Branch Delivery Cost: $1,884,056
📦 Total Annual CBM: 74,743
📦 Handling Cost: $747,432
🏬 Storage Cost: $446,940
🔄 Transfer Cost: $0

🎯 BASELINE TOTAL COST: $3,078,427


---
## VISUALIZE FINAL RESULTS

In [ ]:
# 1. Pie chart cost breakdown
fig_cost = go.Figure(data=[go.Pie(
    labels=['Branch Delivery Cost', 'Handling Cost', 'Storage Cost', 'Transfer Cost'],
    values=[total_branch_delivery_cost, total_handling_cost, total_storage_cost, total_transfer_cost],
    hole=0.4,
    marker_colors=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'],
    textinfo='label+percent+value',
    texttemplate='<b>%{label}</b><br>%{percent}<br>$%{value:,.0f}'
)])

fig_cost.update_layout(
    title="<b>Baseline Cost Breakdown - K-Means Approach (8 Clusters)</b>",
    annotations=[dict(text=f'<b>Total</b><br>${baseline_total_cost:,.0f}', x=0.5, y=0.5,
                     font_size=16, showarrow=False)],
    height=500
)
fig_cost.show()

---
## Summary table by cluster (1 PHASES)

In [ ]:
# 2. Summary table by cluster
cluster_summary_data = []
for result in all_cluster_results:
    cluster_summary_data.append([
        f"Cluster {result['cluster_id']}",
        f"{result['num_shiptos']}",
        f"{result['total_kg']:,.0f} kg",
        f"{result['num_trucks']}",
        f"{result['avg_distance']:.1f} km",
        f"${result['total_cost']:,.0f}"
    ])

fig_table = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>Cluster</b>', '<b>ShipToID</b>', '<b>Total KG</b>', '<b>Trucks</b>', '<b>Avg Distance</b>', '<b>Total Cost</b>'],
        fill_color='lightblue',
        align='left',
        font=dict(size=12, color='white')
    ),
    cells=dict(
        values=list(zip(*cluster_summary_data)),
        fill_color='lavender',
        align='left',
        font=dict(size=11)
    )
)])

fig_table.update_layout(
    title="<b>Results Summary by Cluster - K-Means Approach</b>",
    height=400
)
fig_table.show()